In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets
!pip install gymnasium
import gymnasium as gym
import math
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    pipeline,
)
import torch
from transformers import pipeline
import csv
import datetime

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
resultsFile='/content/drive/MyDrive/Results/BlackJackLlm.csv'
overWriteResultsFile = True

In [ ]:
job_description = """You are a large language model playing blackjack. You can only answer 'hit' or 'stick'"""

In [ ]:
zero_shot_prompt = """you have cards with a sum of #player_sum. #ace_card Dealer has #dealer_card. Should we hit or stick?"""

one_shot_prompt = """When you have cards with a sum lower than 15 and dealer has 10 or higher you should hit.
you have cards with a sum of #player_sum. #ace_card Dealer has #dealer_card. Should we hit or stick?"""

few_shot_prompt = """When you have cards with a sum lower than 15 and dealer has 10 or higher you should hit.
When you have cards with a sum lower than 15 and dealer has 10 or lower you should fold.
you have cards with a sum of #player_sum. #ace_card Dealer has #dealer_card. Should we hit or stick?"""

Chain_of_Thought_prompt = """Q: you have cards with a sum of 10. Dealer has 8. Should we hit or stick?
A: Hit
Q: you have cards with a sum of 14. You have a usable ace. Dealer has 10. Should we hit or stick?
A: Hit
Q: you have cards with a sum of 21. You have a usable ace. Dealer has 8. Should we hit or stick?
A: Stick
Q: you have cards with a sum of #player_sum. #ace_card Dealer has #dealer_card. Should we hit or stick?"""


prompts = {
    "zero_shot_prompt": zero_shot_prompt,
    "one_shot_prompt": one_shot_prompt,
    "few_shot_prompt": few_shot_prompt,
    "Chain_of_Thought_prompt": Chain_of_Thought_prompt
}

In [ ]:
temperatures = [0.1, 0.4, 0.7, 1]

In [ ]:
models_dict = {
    "StableLM Zephyr 3B": "stabilityai/stable-zephyr-3b-dpo"
}

model_id = models_dict["StableLM Zephyr 3B"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, device_map="auto", tokenizer=tokenizer, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [2]:
def generate(question, temperature=0.7, max_new_tokens=512):
    #return 'hit'
    messages = [
        {
            "role": "system",
            "content": job_description,
        },
    ]
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=2, top_p=0.1)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    print(output)
    response_start = output.rfind('<|assistant|>')
    text_output = output[response_start + len('<|assistant|>'):]
    return text_output

In [ ]:
def last_word_used(text):
    # Find the last index of "hit" and "right"
    hit_index = text.rfind("hit")
    stick_index = text.rfind("stick")

    # Compare the indices to determine the last word used
    if hit_index > stick_index:
        return "hit"
    elif stick_index > hit_index:
        return "stick"
    else:
        # Both words have the same index, handle this case as needed
        return "Both words appear at the same position"

In [ ]:
def insertValuesInPrompt(prompt, player_sum, dealer_card, ace):
    prompt = prompt.replace("#player_sum", str(player_sum))
    prompt = prompt.replace("#dealer_card", str(dealer_card))
    if(ace):
      prompt = prompt.replace("#ace_card", "You have a usable ace.")
    else:
      prompt = prompt.replace("#ace_card", "")

    #print(prompt)
    return prompt

In [ ]:
def runExperiment(episodes,promptKey, temperature):
    env = gym.make('Blackjack-v1')#, render_mode="human")
    prompt = prompts[promptKey]
    with open(resultsFile, 'a', newline='') as csvfile:
      # Create a CSV writer object
      csvwriter = csv.writer(csvfile)
      for i_episode in range(episodes):
          observation = env.reset()
          rad = observation[0][2]
          rewards = 0
          randomActions = 0
          hitActions = 0
          stickActions = 0

          player_sum = observation[0][0]
          dealer_card = observation[0][1]
          usable_ace = observation[0][2]

          for t in range(250):
              env.render()

              output = generate(insertValuesInPrompt(prompt, player_sum, dealer_card,usable_ace), temperature, 150)

              action = 0
              #print("OUTPUT")
              #print(output)
              #print("ENDOUTPUT")
              if last_word_used(output.lower()) == "stick":
                  action = 0
                  actionText = 'stick'
                  stickActions+=1
              elif last_word_used(output.lower()) == "hit":
                  action = 1
                  actionText = 'hit'
                  hitActions+=1
              else:
                  actionText = 'random'
                  randomActions+=1
                  action = env.action_space.sample() #Inconclusive answer, random action
              #print(output)
              #print(str(degrees) + " " + direction + " answer: "+actionText+" action: " + str(action))
              observation, reward, terminated, truncated, info = env.step(action)
              rad = observation[2]
              rewards += reward

              player_sum = observation[0]
              dealer_card = observation[1]
              usable_ace = observation[2]

              if terminated:
                  print('prompt: {}, temperature:{}, timesteps: {}, total rewards: {}, hit: {}, stick: {}, random: {}'.format(promptKey, temperature, t+1, rewards,hitActions,stickActions,randomActions))
                  if(overWriteResultsFile):
                    csvwriter.writerow([promptKey, temperature, t+1, rewards, hitActions, stickActions, randomActions,datetime.datetime.now()])
                  break

      env.close()


In [ ]:
#Create csv file
def createNewResultsCsv():
    if(overWriteResultsFile):
        with open(resultsFile, 'w', newline='') as csvfile:
            # Create a CSV writer object
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(["promptKey", "temperature", "timesteps", "rewards", "hitActions", "stickActions", "randomActions", "DateTime"])

In [ ]:
def runExperiments(episodes,prompts, temperatures):
  createNewResultsCsv()
  for promptKey in prompts.keys():
    for temperature in temperatures:
      runExperiment(episodes, promptKey, temperature)

In [ ]:
runExperiments(100,prompts, temperatures)

prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 0, stick: 1, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: 1.0, hit: 0, stick: 1, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: 0.0, hit: 0, stick: 1, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 3, total rewards: -1.0, hit: 3, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 0, stick: 1, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: 1.0, hit: 0, stick: 1, random: 0


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 0, stick: 1, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 3, total rewards: -1.0, hit: 3, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 1, total rewards: -1.0, hit: 1, stick: 0, random: 0
prompt: zero_shot_prompt, temperature:0.1, timesteps: 2, total rewards: -1.0, hit: 1, stick: 1, random: 0
prompt: zero_shot_prompt, temperature:0.1, tim